In [245]:
from __future__ import print_function    # (at top of module)
import warnings
#warnings.filterwarnings('always')
from spotipy.oauth2 import SpotifyClientCredentials
import json
import spotipy
import time
import sys
import csv
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
% matplotlib inline
plt.rcParams['figure.figsize'] = [10, 10]
from matplotlib.pyplot import figure
import math
import seaborn as sns
import config
import requests
import urllib

fileName = "data_600_entries.csv"

In [247]:
data = pd.read_csv(fileName ,encoding='utf-8')
data = data.drop_duplicates(subset=['song_id'], keep='first')
#data = data.drop_duplicates(subset=['song_title'], keep='first')
data = data[data.popularity > 50]
print("Number of entries in original data after cleaning: " + str(len(data.index)))

Number of entries in original data after cleaning: 601


In [248]:
# Build the query from the data
temp_query = ""
youtube_view_counts = []
youtube_video_titles = []
number_of_reaction_videos = 0
# For each row in the dataframe
for index, row in data.iterrows():
    
    # Build the query as song_title + artists
    temp_query += row['song_title'].replace(',', "") + " " + row['artist'].replace('[', "").replace(']', "").replace(',', "")
    #print("===============")
    #print(temp_query)
    
    params_for_query = {"part": "snippet",
                        "maxResults": 5,
                        "order": "relevance",
                        "pageToken": "",
                        "q": temp_query,
                        "key": config.youtube_client_secret,
                        "type": "video",
                        }
    url1 = "https://www.googleapis.com/youtube/v3/search"
    
    # Get video results, select most relevant video (first one)
    # TODO: Still some matching problems - how to get the most relevant video with the most views from official sources?
    page_query = requests.request(method="get", url=url1, params=params_for_query)
    j_results_query = json.loads(page_query.text)
    #print("QUERY ============================")
    max_viewcount = 0
    max_viewcount_title = ""
    for item in j_results_query['items']:
    
        # Print the video title of the first video to match the query string
        #print("Video title: ", item['snippet']['title'])
        video_title = item['snippet']['title']

        # Get the video ID of the first video to match the query string
        #print("Video id: ", item['id']['videoId'])
        video_id = item['id']['videoId']

        # Next, using this video ID we need to get the view count of that video
        params_for_stats = {"part": "statistics",
                            "id": video_id,
                            "key": config.youtube_client_secret,
                           }
        url2 = "https://www.googleapis.com/youtube/v3/videos"
        page_stats = requests.request(method="get", url=url2, params=params_for_stats)
        j_results_stats = json.loads(page_stats.text)

        # Check if problem with retrieving statistics
        if len(j_results_stats['items']) > 0:
            # Get the view count
            #print("Video view count: ", j_results_stats['items'][0]['statistics']['viewCount'])
            view_count  = j_results_stats['items'][0]['statistics']['viewCount']
            if int(view_count) > max_viewcount and row['song_title'].split('(', 1)[0].split('-', 1)[0].rstrip().lower() in video_title.lower():
                max_viewcount = int(view_count)
                max_viewcount_title = video_title
            # Keep count in list
    #print("CHOICE IS: ", max_viewcount_title, " ---- ", max_viewcount)
    youtube_view_counts.append(int(max_viewcount))
    youtube_video_titles.append(max_viewcount_title.replace(",", "").replace('[','(').replace(']', ')'))
    temp_query = ""
    
# Add the two new columns to the data
data['youtube_view_count'] = youtube_view_counts
data['youtube_video_title'] = youtube_video_titles

print()
print("Number of reaction videos found: ", number_of_reaction_videos)
print()
print("Done getting data.")


Number of reaction videos found:  0

Done getting data.


In [218]:
data.head()

,song_id,song_title,artist,popularity,energy,liveness,tempo,speechiness,acousticness,instrumentalness,time_signature,danceability,key,duration,loudness,valence,mode,youtube_view_count,youtube_video_title
0,spotify:track:01IQ4aQgOf0Kkq3a273hmO,Funky Friday,"[Dave, Fredo]",82,0.535,0.1070,125.066,0.1960,0.0783,0.000000,4,0.975,1,182741,-7.227,0.671,1,16292128,Dave - Funky Friday (ft. Fredo)
1,spotify:track:4iZC3zbvagtT4UbvuwH7fQ,Mia Khalifa,[iLOVEFRiDAY],71,0.517,0.3150,145.026,0.1130,0.2300,0.000000,4,0.817,1,177188,-9.613,0.541,1,18316263,iLOVEFRiDAY - Mia Khalifa (TIK TOK ANTHEM) [HI...
2,spotify:track:2Y73798F4zbbbvKT02fbtE,Finalidade Era Ficar em Casa,[MC Kevin o Chris],77,0.978,0.0846,149.888,0.1980,0.5710,0.212000,4,0.810,6,185652,-3.949,0.703,1,4470493,MC Kevin o Chris - Finalidade era ficar em cas...
3,spotify:track:5iwz1NiezX7WWjnCgY5TH4,Calma - Remix,"[Pedro Capó, Farruko]",79,0.773,0.1430,126.899,0.0524,0.3230,0.000000,4,0.826,11,238200,-4.218,0.761,0,57135874,"Pedro Capó, Farruko - Calma (Remix - Official ..."
4,spotify:track:4ej94zqr9xYLbWphCH3TcD,Beauty & Essex (feat. Daniel Caesar & Unknown ...,"[Free Nationals, Daniel Caesar, Unknown Mortal...",65,0.741,0.3340,69.709,0.0542,0.2840,0.000082,4,0.552,11,277046,-5.419,0.718,0,679605,Free Nationals - Beauty & Essex (Audio) (feat....


In [249]:
data.to_csv("data_try_algo_nn.csv", encoding="utf-8", header=True, index=False)